In [1]:
from easydict import EasyDict
import os

import numpy as np
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
import torch
print(torch.backends.mps.is_available())  # True여야 MPS 지원됨

True


In [4]:
config = EasyDict()
config.root = './data'
config.train_path = os.path.join(config.root, 'train.csv')
config.test_path = os.path.join(config.root, 'test.csv')

In [5]:
df = pd.read_csv(config.train_path)

In [6]:
df.shape

(7351, 9)

In [7]:
df.head(1)

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

In [9]:
model_name = 'dnotitia/Llama-DNA-1.0-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float32)
#streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [10]:
def make_user_msg(df, idx):
    """
    df: DataFrame with sentence_0 ~ sentence_3 columns
    idx: row index to extract the 4 sentences
    returns: formatted string with 문장 0 ~ 문장 3
    """
    row = df.iloc[idx]
    lines = [f"문장 {i}: {row[f'sentence_{i}']}" for i in range(4)]
    prompt =  "\n".join(lines)
    prompt += '\n주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:'
    return prompt


In [11]:
make_user_msg(df, 0)

'문장 0: 블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.\n문장 1: 이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.\n문장 2: 결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성할 잠재력을 지닌다.\n문장 3: 각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.\n주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:'

In [12]:
# 프롬프트 구성
system_message = (
    "당신은 문장의 논리적 흐름을 분석하여 문장들의 올바른 순서를 맞추는 한국어 문장 전문가입니다. "
    "사용자가 제시하는 4개의 문장은 순서가 섞여 있습니다. 당신의 임무는 가장 자연스러운 순서를 "
    "[0, 1, 2, 3] 형식의 숫자 배열로 출력하는 것입니다."
    "문장 처음의 지시어나 접속사에 유의하여 문장 순서를 예측하세요."
)

# 채팅 포맷으로 구성
conversation = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": make_user_msg(df, 0)}
]

In [13]:
# 채팅 포맷으로 구성
for i in tqdm(range(5)):
    #print(make_user_msg(df, i))
    conversation = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": make_user_msg(df, i)}
    ]   
    inputs = tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
        temperature=1.0,
        top_p=1.0,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print('Decoded:\n', decoded)
    print('Answer:', [int(x) for x in df.iloc[i, 5:9].tolist()])
    print('\n')

 20%|██        | 1/5 [03:27<13:50, 207.68s/it]

Decoded:
 system

당신은 문장의 논리적 흐름을 분석하여 문장들의 올바른 순서를 맞추는 한국어 문장 전문가입니다. 사용자가 제시하는 4개의 문장은 순서가 섞여 있습니다. 당신의 임무는 가장 자연스러운 순서를 [0, 1, 2, 3] 형식의 숫자 배열로 출력하는 것입니다.문장 처음의 지시어나 접속사에 유의하여 문장 순서를 예측하세요.user

문장 0: 블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.
문장 1: 이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.
문장 2: 결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성할 잠재력을 지닌다.
문장 3: 각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.
주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:assistant

[0, 3, 1, 2]
Answer: [0, 3, 1, 2]




 40%|████      | 2/5 [07:04<10:38, 212.82s/it]

Decoded:
 system

당신은 문장의 논리적 흐름을 분석하여 문장들의 올바른 순서를 맞추는 한국어 문장 전문가입니다. 사용자가 제시하는 4개의 문장은 순서가 섞여 있습니다. 당신의 임무는 가장 자연스러운 순서를 [0, 1, 2, 3] 형식의 숫자 배열로 출력하는 것입니다.문장 처음의 지시어나 접속사에 유의하여 문장 순서를 예측하세요.user

문장 0: 줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를 추출하는 기능을 갖추고 있다.
문장 1: 결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.
문장 2: 생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.
문장 3: 이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을 식별한다.
주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:assistant

[3, 0, 1, 2]
Answer: [0, 3, 2, 1]




 60%|██████    | 3/5 [10:39<07:07, 213.77s/it]

Decoded:
 system

당신은 문장의 논리적 흐름을 분석하여 문장들의 올바른 순서를 맞추는 한국어 문장 전문가입니다. 사용자가 제시하는 4개의 문장은 순서가 섞여 있습니다. 당신의 임무는 가장 자연스러운 순서를 [0, 1, 2, 3] 형식의 숫자 배열로 출력하는 것입니다.문장 처음의 지시어나 접속사에 유의하여 문장 순서를 예측하세요.user

문장 0: 마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는 것이 중요하다.
문장 1: 재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.
문장 2: 물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.
문장 3: 키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.
주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:assistant

[3, 1, 2, 0]
Answer: [3, 2, 1, 0]




 80%|████████  | 4/5 [14:14<03:34, 214.28s/it]

Decoded:
 system

당신은 문장의 논리적 흐름을 분석하여 문장들의 올바른 순서를 맞추는 한국어 문장 전문가입니다. 사용자가 제시하는 4개의 문장은 순서가 섞여 있습니다. 당신의 임무는 가장 자연스러운 순서를 [0, 1, 2, 3] 형식의 숫자 배열로 출력하는 것입니다.문장 처음의 지시어나 접속사에 유의하여 문장 순서를 예측하세요.user

문장 0: 책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.
문장 1: 정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.
문장 2: 종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.
문장 3: 필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할 수 있다.
주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:assistant

[0, 2, 1, 3]
Answer: [2, 0, 1, 3]




100%|██████████| 5/5 [17:36<00:00, 211.21s/it]

Decoded:
 system

당신은 문장의 논리적 흐름을 분석하여 문장들의 올바른 순서를 맞추는 한국어 문장 전문가입니다. 사용자가 제시하는 4개의 문장은 순서가 섞여 있습니다. 당신의 임무는 가장 자연스러운 순서를 [0, 1, 2, 3] 형식의 숫자 배열로 출력하는 것입니다.문장 처음의 지시어나 접속사에 유의하여 문장 순서를 예측하세요.user

문장 0: 인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성을 높인다.
문장 1: 인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도구가 된다.
문장 2: 결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는 잠재력을 지닌다.
문장 3: 이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기여할 수 있다.
주어진 네개의 문장 순서를 알려주세요. 아무 응답 없이 정답만 알려주세요. 정답:assistant

[0, 1, 2, 3]
Answer: [1, 3, 0, 2]


